In [ ]:
# 신경망 학습 순서도 복습
'''
- 전제
신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라고 한다. 신경망 학습은 다음의 4단계로 수행한다.
- 1단계 : 미니배치
훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하고, 그 미니배치의 손실함수 값을 줄이는 것이 목표이다.
- 2단계 : 기울기 산출
미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게하는 방향을 제시한다.
가중치 매개변수의 기울기를 구하면, 매개변수를 조정하였을 때 손실 함수가 어떤 방향으로 바뀔 것인지 알 수 있기 때문이다.
- 3단계 : 매개변수 갱신
가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.
- 4단계 : 반복
1~3단계를 반복한다.
'''

# 이번 챕터에서 다루는 오차역전파법이 등장하는 단계는 2단계인 '기울기 산출'이다.
# 앞 챕터에서는 구현하기 쉬운 수치 미분을 사용했었지만, 이번 챕터에서는 기울기를 효율적이고 빠르게 구할 수 있는 오차역전파법을 사용한다.

In [13]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict() 
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values(): # layers는 OrderedDict이므로, for문을 돌려 순서대로 수행하면 순전파가 계산된다(입력층부터 차례대로 진행되므로)
            x = layer.forward(x)
        
        return x
        
    # x:입력 데이터, t:정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:입력 데이터, t:정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse() # OrderedDict를 거꾸로 뒤집어 역방향 순서 완성
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


In [15]:
# 수치 미분은 느리지만, 구현하기 쉽기 때문에 버그가 있을 확률이 적다.
# 반대로 오차역전파법은 빠르지만, 구현하기 어려워 버그가 있을 확률이 있다.
# 따라서 오차역전파법이 제대로 구현되었는지 검증하기 위해 수치 미분을 사용한다.

import sys, os
sys.path.append(os.pardir)  
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치 차이의 절대값을 구한 후, 그 절댓값들의 평균을 낸다
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(f"{key}:{diff}")
    

# 결과값을 보면, 수치 미분과 오차역전파법으로 구한 기울기의 차이가 아주 작다는 것을 알 수 있다.

W1:5.877397107799665e-10
b1:3.510053137246636e-09
W2:6.014441797446812e-09
b2:1.4002716507838242e-07


In [21]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000 
train_size = x_train.shape[0]
batch_size = 100 
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size) 
    x_batch = x_train[batch_mask] 
    t_batch = t_train[batch_mask] 
    
    # 수치 미분(numerical gradient) 대신 오차 역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch) 
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] 
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"train acc, test acc | {train_acc}, {test_acc}")

train acc, test acc | 0.10583333333333333, 0.1051
train acc, test acc | 0.9043833333333333, 0.9073
train acc, test acc | 0.92205, 0.9248
train acc, test acc | 0.9345666666666667, 0.9353
train acc, test acc | 0.9414666666666667, 0.9394
train acc, test acc | 0.9488166666666666, 0.9475
train acc, test acc | 0.9521833333333334, 0.9485
train acc, test acc | 0.9567666666666667, 0.9543
train acc, test acc | 0.962, 0.9579
train acc, test acc | 0.9643166666666667, 0.9597
train acc, test acc | 0.9667666666666667, 0.962
train acc, test acc | 0.9682833333333334, 0.9624
train acc, test acc | 0.9715333333333334, 0.9655
train acc, test acc | 0.9724333333333334, 0.9649
train acc, test acc | 0.97365, 0.9655
train acc, test acc | 0.9755, 0.9681
train acc, test acc | 0.97535, 0.968


In [22]:
# 이번 장에서 배운 내용

# 계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있다.
# 계산 그래프의 노드는 국소적 계산으로 구성된다. 국소적 계산을 조합해 전체 계산을 구성한다. --> 이 때 국소적 계산은 layer의 forward, backward 메서드로 구현한다.
# 계산 그래프의 순전파는 통상의 계산을 수행한다. 한편, 계산 그래프의 역전파로는 각 노드의 미분을 구할 수 있다.
# 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다(오차역전파법)
# 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지를 확인할 수 있다.(기울기 확인)